In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import netCDF4 as nc
import xarray as xr
import os
from pathlib import Path
from numpy import nanmedian
import scipy
import scipy.signal
import math
from numpy.fft import fft2
import cmocean
import matplotlib
plt.rcParams["figure.figsize"] = 10, 8
%matplotlib inline

In [2]:
current_path = os.getcwd()
current_path

'/home/luisfigueiredo/JUNO/notebooks'

In [3]:

data_folder = os.path.join(current_path,"../data")
data_folder

'/home/luisfigueiredo/JUNO/notebooks/../data'

In [7]:
nc_path = os.path.join(data_folder, "AQUA_MODIS.20220209T133501.L2.SST.NRT.nc")

In [8]:
nc_path

'/home/luisfigueiredo/JUNO/notebooks/../data/AQUA_MODIS.20220209T133501.L2.SST.NRT.nc'

In [10]:

ds = xr.open_dataset(nc_path, group='geophysical_data')
df = ds.to_dataframe()

In [11]:
df

sst  qual_sst  flags_sst  bias_sst  stdv_sst  \
number_of_lines pixels_per_line                                                 
0               0                NaN       4.0        1.0     -18.0     -18.0   
                1                NaN       4.0        1.0     -18.0     -18.0   
                2                NaN       4.0        1.0     -18.0     -18.0   
                3                NaN       4.0        1.0     -18.0     -18.0   
                4                NaN       4.0        1.0     -18.0     -18.0   
...                              ...       ...        ...       ...       ...   
2029            1349             NaN       4.0        1.0     -18.0     -18.0   
                1350             NaN       4.0        1.0     -18.0     -18.0   
                1351             NaN       4.0        1.0     -18.0     -18.0   
                1352             NaN       4.0        1.0     -18.0     -18.0   
                1353             NaN       4.0        1.0     -18.0     -18.0   

                                 sstref  l2_flags  
number_of_lines pixels_per_line                    
0               0                   NaN        50  
                1                   NaN        50  
                2                   NaN        50  
                3                   NaN        50  
                4                   NaN        50  
...                                 ...       ...  
2029            1349               -3.0  33619984  
                1350               -3.0  33619984  
                1351               -3.0  33619984  
                1352               -3.0  33619984  
                1353               -3.0  33619984  

[2748620 rows x 7 columns]

### Código: Load das variaveis MATLAB

In [ ]:

## LOAD das variáveis necessários do netCDF (Este exemplo é com imagens
# MODIS - os caminhos podem ser diferentes)
lon=ncread('AQUA_MODIS.20220209T133501.L2.SST.NRT.nc', '/navigation_data/longitude');
lat=ncread('AQUA_MODIS.20220209T133501.L2.SST.NRT.nc', '/navigation_data/latitude');
sst=ncread('AQUA_MODIS.20220209T133501.L2.SST.NRT.nc', '/geophysical_data/sst');
qual_sst=ncread('AQUA_MODIS.20220209T133501.L2.SST.NRT.nc', '/geophysical_data/qual_sst');

# factor de qualidade para as imagens MODIS - esta linha é para comentar no
# caso do MUR e/ou modelo
sst(qual_sst>1)=NaN;

# Figura com os dados SST - grelha irregular L2
pcolor(lon,lat,sst); shading flat; colorbar

# interpolar dados para uma grelha antes - também não é necessário nos dados L3, L4 ou Modelo

Z1=double(sst);
X1=double(lon);
Y1=double(lat);
X1(isnan(X1))=0;  #caso lon tenha NaN values, estes são convertidos para zeros
Y1(isnan(Y1))=0;  #caso lat tenha NaN values, estes são convertidos para zeros

[X,Y]=meshgrid(-27.296:.02:7.4435,32.2344:0.02:53.7133);

Z=griddata(X1,Y1,Z1,X,Y);

#### Código: LOAD das variaveis em Python (passo a passo)

In [2]:
lon = np.array([1,2,3,4,5,6])
lat = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
sst = np.arange(1,91, dtype=np.double)#.reshape(6,15) 

lon[np.isnan(lon)] = 0    #caso lon tenha NaN values, estes são convertidos para zeros
lat[np.isnan(lat)] = 0    #caso lat tenha NaN values, estes são convertidos para zeros

# corresponde ao [X,Y]=meshgrid(-27.296:.02:7.4435,32.2344:0.02:53.7133); do MATLAB
# No entanto como estamos a usar valores dummy isto para já não serve para nada
#X = np.arange(-27.296, 7.4435, 0.02)
#Y = np.arange(32.2344, 53.7133, 0.02) 
#X, Y = np.meshgrid(X, Y)

#Código de cima mas com dummy variables
X1 = np.linspace(-27.296, 7.4435, 6)
Y1 = np.linspace(32.2344, 53.7133, 15)
X1, Y1 = np.meshgrid(X1, Y1)

from scipy.interpolate import griddata
#Z = griddata([lat, lon], sst, (X1, Y1), method='linear') #NÃO CONSIGO POR ISTO A FUNCIONAR


### Código: Definição da janela MATLAB

In [ ]:
#####################################

# Definição da janela

head(1:2) = [-27.296 7.4435];             #cria variavel head com esses 2 valores nas posiçoes 1 e 2
head(3:4) = [32.2344 53.7133];
z_dim=size(Z); #dimensões da matriz Z
z_actual_range = double([min(Z(:)) max(Z(:))]); #range dos dados
	nx = z_dim(end);		ny = z_dim(end-1);
    node_offset = 0;	
 
	head(5:7) = double([z_actual_range node_offset]);
	head(8) = diff(head(1:2)) / (nx - ~node_offset);
	head(9) = diff(head(3:4)) / (ny - ~node_offset);
    
############################################ 

#### Código: Definição da janela em Python (passo a passo)

In [33]:
head = np.array([-27.296, 7.4435])
head = np.append(head, [32.2344, 53.7133])
#create dummy array Z to test 
Z = np.array([[1,2,3,4,5,6],[1,1,1,1,1,1],[1,2,3,4,5,6],[1,1,1,1,1,1],[1,2,3,4,5,6],
             [1,1,1,1,1,1],[1,2,3,4,5,6],[1,1,1,1,1,10],[1,2,3,4,5,6],[1,1,1,1,1,1],
             [1,2,3,4,5,6],[1,1,1,1,1,1],[1,2,3,4,5,6],[1,1,1,1,1,1],[1,2,3,4,5,6],
              [1,1,1,1,1,1],[1,2,3,4,5,6],[1,1,1,1,1,1],[1,2,3,4,5,6],[1,1,1,1,1,1]])

z_dim = Z.shape    #dimensões da matriz Z

z_actual_range = np.array([np.min(Z[:]), np.max(Z[:])])    #range dos dados
nx = z_dim[1]
ny = z_dim[0]
node_offset = 0

#Solução para o (~node_offset) em MATLAB
if node_offset == 0:
    logical = 1
else:
    logical = 0

head = np.append(head, np.array([z_actual_range[0], z_actual_range[1] , node_offset]))
head = np.append(head, np.array((head[1]- head[0])/(nx - logical)))     
head = np.append(head, np.array((head[3]- head[2])/(ny - logical)))


In [4]:
head

array([-27.296     ,   7.4435    ,  32.2344    ,  53.7133    ,
         1.        ,  10.        ,   0.        ,   6.9479    ,
         2.38654444])

### Código: Definição da parametros MATLAB

In [ ]:
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%  

% Window size 32 × 32

% Window clarity (% of data have to present in the window) 75%
% Overlap 50%

% The ratio of variance between the two populations to the
% variance within the populations θ(τopt )% 0.76

% Gradient ratio 0.9

% Cohesion coefficients [C1, C2, C] [0.9, 0.9, 0.92]
% Mean temperature difference between the two populations (surface waters)
% .25

   %cayula
    minPopProp = 0.20;				% 
	minPopMeanDiff = 0.4;
	minTheta = 0.70;
	minSinglePopCohesion = 0.90;
	minGlobalPopCohesion = 0.70;
   %nieto
%     minPopProp = 0.15;				% 
% 	minPopMeanDiff = 0.4;
% 	minTheta = 0.70;
% 	minSinglePopCohesion = 0.90;
% 	minGlobalPopCohesion = 0.70;
%%%%%%%%%%%%%%%%%%%%%%%

#### Código: Definição da parametros Python (passo a passo)

In [5]:
################################# 

# Window size 32 × 32

# Window clarity (% of data have to present in the window) 75%
# Overlap 50%

# The ratio of variance between the two populations to the
# variance within the populations θ(τopt )% 0.76

# Gradient ratio 0.9

# Cohesion coefficients [C1, C2, C] [0.9, 0.9, 0.92]
# Mean temperature difference between the two populations (surface waters)
# .25

#cayula
minPopProp = 0.20
minPopMeanDiff = 0.4
minTheta = 0.70
minSinglePopCohesion = 0.90
minGlobalPopCohesion = 0.70

#nieto
# minPopProp = 0.15
# minPopMeanDiff = 0.4
# minTheta = 0.70
# minSinglePopCohesion = 0.90
# minGlobalPopCohesion = 0.70

##################################

### Código: 1ª For Loop MATLAB

In [ ]:
%%%%%%%%%%%%%%%%%%%%%%%

[n_rows, n_cols] = size(Z);
	winW16 = 16;	winW32 = 2*16;		winW48 = 3*16;
	s = 0;				% s = 1 means subwindows do NOT share a common border. With s = 0 they do.
	xSide16 = winW16 * head(8);
	ySide16 = winW16 * head(9);
	xSide32 = (winW32 - s) * head(8);			% X and Y sizes of the 32x32 window in real coordinates
	ySide32 = (winW32 - s) * head(9);			% -1 because coords are in grid registration
	nWinRows = floor(n_rows / winW16);		nWinCols = floor(n_cols / winW16);
	for (wRow = 1:nWinRows-2)
		r1 = (wRow - 1) * winW16 + 1;		r2 = r1 + winW48 - s;	% Start and stop indices and coords of current window
		y0 = head(3) + (wRow-1) * ySide16;				%y1 = y0 + (winW48-1) * head(9);
		for (wCol = 1:nWinCols-2)
			c1 = (wCol - 1) * winW16 + 1;	c2 = c1 + winW48 - s;
			x0 = head(1) + (wCol-1) * xSide16;			%x1 = x0 + (winW48-1) * head(8);
			wPad = Z(r1:r2, c1:c2);						% 48x49 (or 48x48 if s == 1) Window
			% |4|3|
			% |1|2|
			% Rows and columns of the 4 sub-windows (avoids a double loop). We loop along the order of the
			% above sketch. There will be a common region when we slide the 33x33 window with shifts of 16
			% When we at 1 (a 33x33 window) that common zone is depicted at position 1 form the sketch below,
			% and so forth for the other three positions.
			rr = [1 1 2 2];		cc = [1 2 2 1];

			% Set the index of the four corner windows of 17x17 representing each the same physical region
			% when we loop 4 times the sliding window of 33x33 in the larger 49x49 window (above sketch).
			% |2|1|
			% |3|4|
			if (s == 1),	corners = [17 32 17 32; 17 32 1 16; 1 16 1 16; 1 16 17 32];		% Less good
			else			corners = [17 33 17 33; 17 33 1 17; 1 17 1 17; 1 17 17 33];
			end

			for (k = 1:4)					% Loop over the 4 sliding 32x32 sub-windows of the larger 48x48 one.
				m1 = (rr(k) - 1) * winW16 + 1;			m2 = m1 + 2*winW16 - s;		% indices of the sliding 33x33 window
				n1 = (cc(k) - 1) * winW16 + 1;			n2 = n1 + 2*winW16 - s;
				w = double((wPad(m1:m2, n1:n2)));					% Sub-window with size 33x33
				subWinX0 = x0 + (cc(k)-1) * xSide16;	subWinX1 = subWinX0 + xSide32;	% Corner coordinates
				subWinY0 = y0 + (rr(k)-1) * ySide16;	subWinY1 = subWinY0 + ySide32;
				R = [subWinX0 subWinX1 subWinY0 subWinY1];
				[x, y, z, exitType] = getFrontInWindow(w, R, minTheta, minPopProp, minPopMeanDiff, ...
													   minSinglePopCohesion, minGlobalPopCohesion, corners(k,:));
				if (~exitType)       #terminates MATLAB program (same as quit)
					h=line('XData',x, 'YData',y, 'userdata',z, 'Tag','SSTfront');
					%rectX = [subWinX0 subWinX0 subWinX1 subWinX1 subWinX0];
					%rectY = [subWinY0 subWinY1 subWinY1 subWinY0 subWinY0];
					%hl = line('XData',rectX, 'YData',rectY, 'Parent',hMirHand.axes1);
					%hl = line('XData',[x0 x0 x1 x1 x0], 'YData',[y0 y1 y1 y0 y0], 'Parent',hMirHand.axes1);
					%draw_funs(hl,'line_uicontext')		% Set lines's uicontextmenu
				end
			end

		end
		drawnow
    end
% ---------------------------------------------------------------------------------------------------


#### Código: 1ª For Loop Python (passo a passo)

In [24]:
[n_rows, n_cols] = Z.shape
winW16 = 16
winW32 = 16*2
winW48 = 16*3

s=0 #s=1 means subwindows do NOT share a common border. With s = 0 they do.

xSide16 = winW16*head[7]
ySide16 = winW16*head[8]
xSide32 = (winW32 - s) * head[7]
ySide32 = (winW32 - s) * head[8]

nWinRows = math.floor(n_rows/winW16)
nWinCols = math.floor(n_cols/winW16)

wRow=1
wCol=1
for wRow in range(1, nWinRows-3):
    r1 = (wRow-1) * winW16 + 1;
    r2 = r1 + winW48 -s     #start and stop indices and coords of current window
    y0 = head[2] + (wRow-1)*ySide    #y1 = y0 + (winW48-1)*head[8]
    
    for wCol in range(1, nWinCols-3):
        c1 = (wCol - 1)*winW16+1
        c2 = c1 + winW48 - s
        x0 = head[0] + (wCol-1) * xSide16     #x1 = x0 + (winW48-1)*head[7]
        wPad = Z[r1-1:r2, c1-1:c2]
        
        rr = np.array([1,1,2,2])
        cc = np.array([1,2,2,1])
        
        if s==1:
            corners = np.array([[17, 32, 17, 32], [17, 32, 1, 16], [1, 16, 1, 16], [1, 16, 17, 32]])  #less good
        else:
            corners = np.array([[17, 33, 17, 33], [17, 33, 1, 17], [1, 17, 1, 17], [1, 17, 17, 33]])
            
        for k in range(1:5):     #loop over the 4 slidding 32X32 sub-windows of the larger 48x48 one
            m1 = (rr[k] - 1) * winW16 + 1
            m2 = m1 + 2 * winW16 - s             #indices of the slidding 33X33 window
            ni = (cc[k] - 1) * winW16 + 1
            n2 = n1 + 2 * winW16 - s
            
            w = wPad[m1-1:m2, n1-1:n2].astype('double')      #sub window with size 33x33
            
            subWinX0 = X0 + (cc[k] - 1) * xSide16
            subWinX1 = subWinX0 + xSide32
            
            subWinY0 = y0 + (rr[k] - 1) * ySide16
            subWinY1 = subWinY0 + ySide32
            
            R = np.array([subWinX0, subWinX1, subWinY0, subWinY1])
        

In [31]:
n_cols

6

In [40]:
w_PAD = Z[4:10, 2:5].astype('double')

In [41]:
w_PAD

array([[3., 4., 5.],
       [1., 1., 1.],
       [3., 4., 5.],
       [1., 1., 1.],
       [3., 4., 5.],
       [1., 1., 1.]])

### Código: function getFrontInWindow() MATLAB

In [ ]:
%% função especifica. Pode ficar noutro ficheiro e ser chamada
function [xdata, ydata, z, exitType] = getFrontInWindow(w, head, minTheta, minPopProp, minPopMeanDiff, ...
                                       minSinglePopCohesion, minGlobalPopCohesion, corners)
% ...
% If CORNERS exist than it holds the start and stop index of 1/4 sub-window of W inside which
% we will retain the fronts found. Detections outside this sub-window are ignored
%
% Here we follow, patially, what is donne in MGET (FrontUtils.cpp by Jason Roberts) that in turn,
% according to program's notes, followed an original Fortran code by Dave Ulman.

	if (nargin < 8),	corners = [];	end
	xdata = [];		ydata = [];		z = [];		exitType = 0;

	mask = isnan(w);	
	haveNaNs = any(mask(:));
	n_NaNs = 0;
	if (haveNaNs)
		n_NaNs = sum(mask(:));
		if (n_NaNs / numel(w) > 0.5)			% Window must be at least > half filled
			exitType = -1;
			return
		end
	end

	mi_ma = double([min(w(:)) max(w(:))]);		
	n = ceil(diff(mi_ma) / 0.02);
% 	[y,xout] = histo_m('hist',w(:), n, mi_ma); o hist funciona
    [y,xout] = hist(w(:),n,mi_ma);  % esta é a função histograma do MATLAB. deve haver algo parecido no python

	thresValue = xout(1);	totalCount = numel(w) - n_NaNs;
	threshPopACount = 0;	threshSeparation = -1;
	threshPopAMean = 0;		threshPopBMean = 0;

	w(mask) = 0;				% Replace NaNs with 0's
	totalSum = sum(w(:));
	totalSumSquares = sum(w(:) .* w(:));

	for (k = 2:n-1)				% Ignore first and last as candidates
		popASum = sum(y(1:k) .* xout(1:k));
		popBSum = sum(y(k+1:end) .* xout(k+1:end));
		popACount = sum(y(1:k));
		popBCount = sum(y(k+1:end));

		popAMean = popASum / popACount;
		popBMean = popBSum / popBCount;
		separation = popACount * popBCount * (popAMean - popBMean) * (popAMean - popBMean);
		if (separation > threshSeparation)
			threshSeparation = separation;
			thresValue = xout(k);
			threshPopACount = popACount;
			threshPopAMean = popAMean;
			threshPopBMean = popBMean;
		end
	end

	% Continue only if the proportional size of the smaller population exceeds the minimum allowed value.
	% This test corresponds to equation 14 in Cayula-Cornillon
	if (threshPopACount / totalCount < minPopProp)
		exitType = 1;
		return
	end
	if (1.0 - threshPopACount / totalCount < minPopProp)
		exitType = 1;
		return
	end
	% Abort this window if the difference in the populations' means is less than a minimum value.
	if (threshPopBMean - threshPopAMean < minPopMeanDiff)
		exitType = 2;
		return
	end

	% Calculate the criterion function THETA(TAUopt) discussed on page 72 of the paper.
	totalMean = totalSum / totalCount;
	variance = totalSumSquares - (totalMean * totalMean * totalCount);
	theta = threshSeparation / (variance * totalCount);
	if (theta < minTheta)
		exitType = 3;
		return
	end

	% Cohesion
	% Count the number of times a population A cell is immediately adjacent to another population A cell,
	% and the same for population B. A cell can be adjacent on four sides. Count only two of them
	% (bottom and right side) because doing all four would be redundant. Do not count diagonal neighbors.
	countANextToA = 0;		countBNextToB = 0;
	countANextToAOrB = 0;	countBNextToAOrB = 0;
	[n_rows, n_cols] = size(w);
	for (col = 1:n_cols-1)
		for (row = 1:n_rows-1)
			if (haveNaNs && (mask(row,col) || mask(row+1,col) || mask(row, col + 1)))
				continue
			end
			% Examine the bottom neighbor
			if (w(row,col) <= thresValue)
				countANextToAOrB = countANextToAOrB + 1;
				if (w(row + 1, col) <= thresValue)
					countANextToA = countANextToA + 1;
				end
			else
				countBNextToAOrB = countBNextToAOrB + 1;
				if (w(row + 1, col) > thresValue)
					countBNextToB = countBNextToB + 1;
				end
			end
			% Examine the right neighbor
			if (w(row,col) <= thresValue)
				countANextToAOrB = countANextToAOrB + 1;
				if (w(row, col + 1) <= thresValue)
					countANextToA = countANextToA + 1;
				end
			else
				countBNextToAOrB = countBNextToAOrB + 1;
				if (w(row, col + 1) > thresValue)
					countBNextToB = countBNextToB + 1;
				end
			end

		end
	end
	
	popACohesion = countANextToA / countANextToAOrB;
	popBCohesion = countBNextToB / countBNextToAOrB;
	globalCohesion = (countANextToA + countBNextToB) / (countANextToAOrB + countBNextToAOrB);
	if (popACohesion < minSinglePopCohesion)
		exitType = 4;
		return
	end
	if (popBCohesion < minSinglePopCohesion)
		exitType = 4;
		return
	end
	if (globalCohesion < minGlobalPopCohesion)
		exitType = 4;
		return
	end

	% OK, if we reach here we have a front. Compute its contour. This part is not discussed in any
	% of the papers I saw and is different from the aproach followed in MGET.
	X = linspace(head(1),head(2), n_cols);
	Y = linspace(head(3),head(4), n_rows);
	if (isempty(corners))		% Use the full window
		w = double(w);			% It has to be because of the contourc
		if (haveNaNs)
			w(w == 0) = NaN;	% Need to restore the NaNs to not invent new contours around zeros
		end
		c = contourc(X,Y,w,[thresValue thresValue]);
	else
		% The four corners have these indices [17 32 17 32; 17 32 1 16; 1 16 1 16; 1 16 17 32]
		% and the variable corners has one of its rows (the current to be retained sub-window)
		X = X(corners(3):corners(4));
		Y = Y(corners(1):corners(2));
		w = double(w(corners(1):corners(2), corners(3):corners(4)));
		if (haveNaNs)
			w(w == 0) = NaN;
		end
		if (~all(all(isnan(w))))    % Sometimes w is all NaNs, which would screw later.
			c = contourc(X,Y,w,[thresValue thresValue]);
		else
			c = [];
		end
	end
	limit = size(c,2);
	i = 1;
	while(i < limit)
		npoints = c(2,i);
		x = c(1,i+1:i+npoints);		y = c(2,i+1:i+npoints);
		if (x(1) == x(end) || npoints < 7)		% We don't want closed lines nor very short ones
			i = i+npoints+1;
			continue
		end
		if (isempty(xdata))			% First contour
			xdata = x;	ydata = y;
		else
			xdata = [xdata NaN x];
			ydata = [ydata NaN y];
		end
		i = i+npoints+1;
		%xdata = c(1,2:npoints+1);		ydata = c(2,2:npoints+1);
	end
	z = thresValue;
	
	if (isempty(xdata))
		exitType = 5;
    end 
end

### Código: function getFrontInWindow() in Python (passo a passo)

In [ ]:
def getFrontInWindow(w, head, minTheta, minPopProp, minPopMeanDiff, minSinglePopCohesion, 
                     minGlobalPopCohesion, corners):
    
    if nargin<8:
        corners = []
        
    xdata=[]
    ydata=[]
    z=[]
    #exitType=0
    
    mask = np.isnan(w).astype('int')   #if is nan convert to 1 if its not nan convert to 0
    haveNaNs = np.any(mask[:]).astype('int')
    n_NaNs=0
    
    if haveNaNs:
        n_NaNs = sum(mask.flatten()[:])
        if (n_NaNs/len(w.flatten())>0.5):     #window must be at least > half filled
            exitType=-1
            break
            return
        
    mi_ma = [np.nanmin(w), np.nanmax(w)]
    n = math.ceil((mi_ma[1]-mi_ma[0])/0.02)
    [y, xout] = np.histogram(w[:], n, mi_ma)    #y->frequency counts, Xout->bin location
    #xout = xout[:-1]     #len(xout) is 1 more than len(y) so i will drop the last element of xout
    
    thresValue = xout[0]
    totalCount = len(w.flatten()) - n_NaNs
    threshPopACount = 0
    threshSeparation = -1
    threshPopAMean = 0
    threshPopBMean = 0
    
    w[mask] =0     #Replace NaNs with 0's
    totalSum = sum(w.flatten())
    totalSumSquares = sum(w.flatten()*w.flatten())
    
    for k in range(1,n-1):     #ignore the first and last candidates (senão seria de 0 a n)
        popASum = sum(y[0:k] * xout[0:k])
        popBSum = sum(y[k+1:] * xout[k+1:])
        
        
        
        
        

In [419]:
mask = np.array([[0, 0, 0, 1, 0, 1, 0, 0, 1], [0, 1, 0, 1, 0, 0, 1, 0, 1], [0, 0, 0, 1, 0, 1, 0, 0, 1]], dtype='bool')

In [420]:
w = np.array([[1,2,3,4,5,np.nan,1,1,2], [1,2,3,4,0,6,0,1,2], [1,2,3,4,10,6,0,1,2]])

In [421]:
        
xdata=[]
ydata=[]
z=[]
#exitType=0
    
#mask = np.isnan(w).astype('int')   #if is nan convert to 1 if its not nan convert to 0
haveNaNs = np.any(mask[:]).astype('int')
n_NaNs=0
    
#if haveNaNs:
n_NaNs = sum(mask.flatten()[:])
 #   if (n_NaNs/len(w.flatten())>0.5):     #window must be at least > half filled
        #exitType=-1
  #      break
   #     return
        
mi_ma = [np.nanmin(w), np.nanmax(w)]
n = math.ceil((mi_ma[1]-mi_ma[0])/0.02)
[y, xout] = np.histogram(w[:], n, mi_ma)    #y->frequency counts, Xout->bin location 
xout = xout[:-1]     #len(xout) is 1 more than len(y) so i will drop the last element of xout
    
thresValue = xout[0]
totalCount = len(w.flatten()) - n_NaNs
threshPopACount = 0
threshSeparation = -1
threshPopAMean = 0
threshPopBMean = 0
    
w[mask] =0     #Replace NaNs with 0's
totalSum = sum(w.flatten())
totalSumSquares = sum(w.flatten()*w.flatten())
    
for k in range(1,n-1):     #ignore the first and last candidates (senão seria de 0 a n)
    popASum = sum(y[0:k] * xout[0:k])
    popBSum = sum(y[k+1:] * xout[k+1:])
    popACount = sum(y[0:k])
    popBCount = sum(y[k+1:])
    
    popAMean = popASum/popACount
    popBMean = popBSum/popBCount
    separation = popACount * popBCount * (popAMean - popBMean) * (popAMean - popBMean)
    if separation>threshSeparation:
        threshSeparation = separation
        thresValue = xout[k]
        threshPopACount = popACount
        threshPopAMean = popAMean
        threshPopBMean = popBMean

In [423]:
popAMean = popASum/popACount
popAMean

2.28

In [410]:
popBMean = popBSum/popBCount
popBMean

9.98

In [422]:
y[0]

3

In [425]:
xout[k+1]

9.98

In [312]:
mask.flatten()

array([False, False, False,  True, False,  True, False, False,  True,
       False,  True, False,  True, False, False,  True, False,  True,
       False, False, False,  True, False,  True, False, False,  True])

array([[False, False, False,  True, False,  True, False, False,  True],
       [False,  True, False,  True, False, False,  True, False,  True],
       [False, False, False,  True, False,  True, False, False,  True]])